# HMTK Geological Tools Demonstration

This notepad demonstrates the use of the HMTK geological tools for preparing fault source models for input into OpenQuake

## Construction of the Geological Input File

An active fault model input file contains two sections:

1) A tectonic regionalisation - this can provide a container for a set of properties that may be assigned to multiple faults by virtue of a common tectonic region

2) A set of active faults

#### Tectonic Regionalisation Representation in the Fault Source File

In the tectonic regionalisation information each of the three properties can be represented according to a set of weighted values.
For example, in the case below faults in an arbitrarily named tectonic region (called here "GEM Region 1") will share the same set
of magnitude scaling relations and shear moduli, unless over-written by the specific fault. Those faults assigned to "GEM Region 2"
will have the magnitude scaling relation fixed as WC1994 and the shear modulus of 30 GPa

### Active Fault Model

A set of active faults will be defined with a common ID and name. 

An active fault set containing a single fault is shown below:

#### Fault Geometry Representations - Example 1: Simple Fault

#### Fault Geometry Representations - Example 2: Complex Fault

#### Rupture Properties

The rupture requires characterisation of the rake (using the Aki & Richards 2002 convention), the slip-type, the slip completeness factor
(an integer constraining the quality of the slip information with 1 being the hights quality), the range of slip values and their 
corresponding weights, and the aseismic slip coefficient (the proportion of slip released aseismically, 1.0 - coupling coefficient)

In [1]:
input_dir = r'C:\Users\AMH-L91\Documents\Projects\openquake-notebooks\input_data' # directory containing yaml files
output_dir = r'C:\Users\AMH-L91\Documents\Projects\openquake-notebooks\output_data' # output directory

Select TRT (ASC, SZ Interface, or SZ Intraslab):

In [2]:
trt = 'SZ Intraslab'

## The Magnitude Frequency Distributions

In [3]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings('ignore')

In [4]:
#Import tools
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from openquake.hmtk.plotting.faults.geology_mfd_plot import plot_recurrence_models
from openquake.hazardlib.scalerel.wc1994 import WC1994  
from openquake.hazardlib.scalerel.strasser2010 import (StrasserInterface, 
StrasserIntraslab)

print ("Everything imported OK!")

Everything imported OK!


The following examples refer to a fault with the following properties:

Length (Along-strike) = 100 km,
Width (Down-Dip) = 20 km,
Slip = 10.0 mm/yr,
Rake = 0. (Strike Slip),
Magnitude Scaling Relation = Wells & Coppersmith (1994),
Shear Modulus = 30.0 GPa

# Set up fault parameters
slip = 10.0  # Slip rate in mm/yr

# Area = along-strike length (km) * down-dip with (km)
area = 100.0 * 20.0

# Rake = 0.
rake = 0.

# Magnitude Scaling Relation
msr = WC1994()

### Anderson & Luco (Arbitrary)

This describes a set of distributons where the maximum magnitude is assumed to rupture the whole fault surface

#Magnitude Frequency Distribution Example

anderson_luco_config1 = {'Model_Name': 'AndersonLucoArbitrary',
                         'Model_Type': 'First',
                         'Model_Weight': 1.0,  # Weight is a required key - normally weights should sum to 1.0 - current example is simply illustrative! 
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config2 = {'Model_Name': 'AndersonLucoArbitrary',
                         'Model_Type': 'Second',
                         'Model_Weight': 1.0,
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config3 = {'Model_Name': 'AndersonLucoArbitrary',
                         'Model_Type': 'Third',
                         'Model_Weight': 1.0,   
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
# Create a list of the configurations
anderson_luco_arb = [anderson_luco_config1, anderson_luco_config2, anderson_luco_config3]

# View the corresponding magnitude recurrence model
plot_recurrence_models(anderson_luco_arb, area, slip, msr, rake, msr_sigma=0.0)

### Anderson & Luco (Area - MMax)

This describes a set of distributons where the maximum rupture extent is limited to only part of the fault surface

anderson_luco_config1 = {'Model_Name': 'AndersonLucoAreaMmax',
                         'Model_Type': 'First',
                         'Model_Weight': 1.0,  # Weight is a required key - normally weights should sum to 1.0 - current example is simply illustrative! 
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config2 = {'Model_Name': 'AndersonLucoAreaMmax',
                         'Model_Type': 'Second',
                         'Model_Weight': 1.0,
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config3 = {'Model_Name': 'AndersonLucoAreaMmax',
                         'Model_Type': 'Third',
                         'Model_Weight': 1.0,   
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}

# For these models a displacement to length ratio is needed
disp_length_ratio = 1.25E-5

# Create a list of the configurations
anderson_luco_area_mmax = [anderson_luco_config1, anderson_luco_config2, anderson_luco_config3]

# View the corresponding magnitude recurrence model
plot_recurrence_models(anderson_luco_area_mmax, area, slip, msr, rake, msr_sigma=0.0)


### Characteristic Earthquake

The following example illustrates a "Characteristic" Model, represented by a Truncated Gaussian Distribution

characteristic = [{'Model_Name': 'Characteristic',
                   'MFD_spacing': 0.05,
                   'Model_Weight': 1.0,
                   'Maximum_Magnitude': None,
                   'Sigma': 0.15,  # Standard Deviation of Distribution (in Magnitude Units) - omit for fixed value
                   'Lower_Bound': -3.0,   # Bounds of the distribution correspond to the number of sigma for truncation
                   'Upper_Bound': 3.0}]

# View the corresponding magnitude recurrence model
plot_recurrence_models(characteristic, area, slip, msr, rake, msr_sigma=0.0)

### Youngs & Coppersmith (1985) Models

The following describes the recurrence from two distributions presented by Youngs & Coppersmith (1985): 1) Exponential Distribution, 2) Hybrid Exponential-Characteristic Distribution

exponential = {'Model_Name': 'YoungsCoppersmithExponential',
               'MFD_spacing': 0.1,
               'Maximum_Magnitude': None,
               'Maximum_Magnitude_Uncertainty': None,
               'Minimum_Magnitude': 5.0,
               'Model_Weight': 1.0,
               'b_value': [0.8, 0.1]}

hybrid = {'Model_Name': 'YoungsCoppersmithCharacteristic',
          'MFD_spacing': 0.1,
          'Maximum_Magnitude': None,
          'Maximum_Magnitude_Uncertainty': None,
          'Minimum_Magnitude': 5.0,
          'Model_Weight': 1.0,
          'b_value': [0.8, 0.1],
          'delta_m': None}

youngs_coppersmith = [exponential, hybrid]

# View the corresponding magnitude recurrence model
plot_recurrence_models(youngs_coppersmith, area, slip, msr, rake, msr_sigma=0.0)


## Epistemic Uncertainty Examples

This example considers the fault defined at the top of the page. This fault defines two values of slip rate and two different magnitude frequency distributions

In [5]:
def show_file_contents(filename):
    """
    Shows the file contents
    """
    fid = open(filename, 'r')
    for row in fid.readlines():
        print(row)
    fid.close()

input_coll = {
    'ASC': 'Simple_Faults_ASC.toml',
    'SZ Interface': 'Simple_Faults_Interface.toml',
    'SZ Intraslab': 'Simple_Faults_Intraslab.toml'
}

input_file = os.path.join(input_dir, input_coll[trt])
show_file_contents(input_file)

Fault_Model_ID = 3

Fault_Model_Name = "Faults Intraslab"

[[Fault_Model]]

ID = 20

Tectonic_Region = "Subduction Intraslab"

Fault_Name = "Mid Manila Trench 1"

Rake = 95.0

Slip_Type = "Reverse"

Slip_Completeness_Factor = 1.0

Aseismic = 0.99

Aspect_Ratio = 1.5

[[Fault_Model.MFD_Model]]

Model_Name = "YoungsCoppersmithCharacteristic"

MFD_spacing = 0.1

Model_Weight = 0.5

Minimum_Magnitude = 4.8

Maximum_Magnitude = 8.0

b_value = [ 0.924, 0.359,]



[[Fault_Model.MFD_Model]]

Model_Name = "AndersonLucoArbitrary"

Model_Type = "First"

MFD_spacing = 0.1

Model_Weight = 0.5

Maximum_Magnitude = 8.0

Minimum_Magnitude = 4.8

b_value = [ 0.924, 0.359,]





[Fault_Model.Fault_Geometry]

Fault_Typology = "Simple"

Fault_Trace = [ 120.056488139794, 20.4888291113274, 120.248077953476, 19.9054132051134, 120.309941095562, 19.5695571601802, 120.219605180908, 19.3186725607661, 120.001607051998, 19.0850858261459, 119.733040431811, 18.7801639580133, 119.636256620232, 18.3047696380102, 119.4

### Example 1 - Full Enumeration

In this example each individual MFD for each branch is determined. In the resulting file the fault is duplicated n_branches number of times, with the
corresponding MFD multiplied by the end-branch weight

In [6]:
# Import the Parser
from openquake.hmtk.parsers.faults.fault_yaml_parser import FaultYmltoSource

# Fault mesh discretization step
mesh_spacing = 2.5 #(km)

# Read in the fault model
reader = FaultYmltoSource(input_file)
fault_model, tectonic_region = reader.read_file(mesh_spacing)

# Construct the fault source model (this is really running the MFD calculation code)
fault_model.build_fault_model()

# Write to an output NRML file
output_full_coll = {
    'ASC': 'Simple_Faults_ASC_Full.xml',
    'SZ Interface': 'Simple_Faults_Interface_Full.xml',
    'SZ Intraslab': 'Simple_Faults_Intraslab_Full.xml'
}

output_file_1 = os.path.join(output_dir, output_full_coll[trt])

fault_model.source_model.serialise_to_nrml(output_file_1)

show_file_contents(output_file_1)

<?xml version="1.0" encoding="utf-8"?>

<nrml

xmlns="http://openquake.org/xmlns/nrml/0.5"

xmlns:gml="http://www.opengis.net/gml"

>

    <sourceModel

    name="Faults Intraslab"

    >

        <sourceGroup

        rup_interdep="indep"

        src_interdep="indep"

        tectonicRegion="Subduction Intraslab"

        >

            <simpleFaultSource

            id="028_1"

            name="Mid Manila Trench 3"

            >

                <simpleFaultGeometry>

                    <gml:LineString>

                        <gml:posList>

                            119.1133 14.0645 119.3219 13.7775 119.3266 13.6444 119.5984 13.4902 119.6381 13.3222 119.9075 13.1154 120.1622 12.9748 120.4942 12.9384 120.6722 12.7475 120.8709 12.641

                        </gml:posList>

                    </gml:LineString>

                    <dip>

                        60.24771487

                    </dip>

                    <upperSeismoDepth>

                        50.0

     

### Example 2: Collapsed Branches

In the following example we implement the same model, this time collapsing the branched. This means that the MFD is discretised and the incremental rate
in each magnitude bin is the weighted sum of the rates in that bin from all the end branches of the logic tree.

When collapsing the branches, however, it is necessary to define a single Magnitude Scaling Relation that will need to be assigned to the fault for
use in OpenQuake.

In [7]:
# Read in the fault model
reader = FaultYmltoSource(input_file)
fault_model, tectonic_region = reader.read_file(mesh_spacing)

# Scaling relation for export
msr_coll = {
    'ASC': WC1994(),
    'SZ Interface': StrasserInterface(),
    'SZ Intraslab': StrasserIntraslab()
}
output_msr = msr_coll[trt]

# Construct the fault source model - collapsing the branches
fault_model.build_fault_model(collapse=True, rendered_msr=output_msr)

# Write to an output NRML file
output_collapsed_coll = {
    'ASC': 'Simple_Faults_ASC_Collapsed.xml',
    'SZ Interface': 'Simple_Faults_Interface_Collapsed.xml',
    'SZ Intraslab': 'Simple_Faults_Intraslab_Collapsed.xml'
}
output_file_2 = os.path.join(output_dir, output_collapsed_coll[trt])

fault_model.source_model.serialise_to_nrml(output_file_2)

show_file_contents(output_file_2)

<?xml version="1.0" encoding="utf-8"?>

<nrml

xmlns="http://openquake.org/xmlns/nrml/0.5"

xmlns:gml="http://www.opengis.net/gml"

>

    <sourceModel

    name="Faults Intraslab"

    >

        <sourceGroup

        rup_interdep="indep"

        src_interdep="indep"

        tectonicRegion="Subduction Intraslab"

        >

            <simpleFaultSource

            id="028_1"

            name="Mid Manila Trench 3"

            >

                <simpleFaultGeometry>

                    <gml:LineString>

                        <gml:posList>

                            119.1133 14.0645 119.3219 13.7775 119.3266 13.6444 119.5984 13.4902 119.6381 13.3222 119.9075 13.1154 120.1622 12.9748 120.4942 12.9384 120.6722 12.7475 120.8709 12.641

                        </gml:posList>

                    </gml:LineString>

                    <dip>

                        60.24771487

                    </dip>

                    <upperSeismoDepth>

                        50.0

     

### Example 3: Gulf of Corinth Faults

In the last example we will take some real fault data from the Gulf of Corinth

# For plotting
from openquake.hmtk.plotting.mapping import HMTKBaseMap
corinth_fault_file = "input_data/GulfOfCorinth_Faults.yml"

reader = FaultYmltoSource(corinth_fault_file)
corinth_faults, tectonic_region = reader.read_file(mesh_spacing)

print "           ID                         Name   Area (km^2)"
for i, fault in enumerate(corinth_faults.faults):
    print "%2g    %s    %25s    %10.3f" % (i, fault.id, fault.name, fault.area)

For convenience we will collapse the branches again

# Scaling relation for export
output_msr = WC1994()

# Construct the fault source model - collapsing the branches
corinth_faults.build_fault_model(collapse=True, rendered_msr=output_msr)

Let's take a look at the fault sources

map_config = {'min_lon': 20.9, 'max_lon': 24.0,
              'min_lat': 37.5, 'max_lat': 39.0, 'resolution':'h'}

# Map the Source
src_basemap = HMTKBaseMap(map_config, "Gulf of Corinth Faults")
src_basemap.add_source_model(corinth_faults.source_model)

##### Find and Show Events within 30 km of the Xylocastro Fault

from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueParser
from openquake.hmtk.seismicity.selector import CatalogueSelector

# Load in the catalogue
catalogue_file = "input_data/Aegean_ExtendedCat1.csv"
parser = CsvCatalogueParser(catalogue_file)
catalogue = parser.read_file()
print catalogue.get_number_events()

#Create a selector object
selector = CatalogueSelector(catalogue)
xylocastro_fault = corinth_faults.faults[15]

# Select earthquakes within 30 km (rupture distance) of the fault
xylocastro_fault.select_catalogue(selector, 30.0, distance_metric="rupture")
print "%g events with 30 km of the Xylocastro Fault" % xylocastro_fault.catalogue.get_number_events()

Show the events

map_config = {'min_lon': 20.9, 'max_lon': 24.0,
              'min_lat': 37.5, 'max_lat': 39.0, 'resolution':'h'}

# Map the Source
src_basemap = HMTKBaseMap(map_config, "Gulf of Corinth Faults")
src_basemap.add_source_model(corinth_faults.source_model, overlay=True)
src_basemap.add_catalogue(xylocastro_fault.catalogue)

### Export output to XML File

corinth_output_file = "output_data/New_Gulf_Of_Corinth_Faults.xml"
corinth_faults.source_model.serialise_to_nrml(corinth_output_file)
show_file_contents(corinth_output_file)